# Data wrangling with pandas 1
튜토리얼은 아래의 목차로 구성되어있다.
(본 튜토리얼은 "파이썬 라이브러리를 활용한 데이터 분석"을 기초로 만들어졌다.)

---

1. 데이터 합치기
    * 1.1 데이터베이스 스타일로 DataFrame 합치기
    * 1.2 색인 머지하기
    * 1.3 축 따라 이어붙이기
    * 1.4 겹치는 데이터 합치기
2. 재형성과 피벗
    * 2.1 계층적 색인으로 재형성하기
    * 2.2 피버팅으로 데이터 나열 방식 바꾸기
3. 데이터 변형
    * 3.1 중복 제거하기
    * 3.2 함수나 매핑 이용해 데이터 변형하기
    * 3.3 값 치환하기
    * 3.4 축 색인 이름 바꾸기
    * 3.5 개별화와 양자화
    * 3.6 특이값 찾아내고 제외하기
    * 3.7 치환과 임의 샘플링
    * 3.8 표시자 / 더미 변수
4. 문자열 다루기
    * 4.1 문자열 객체 메서드
    * 4.2 정규표현식
    * 4.3 pandas의 벡터화된 문자열 함수

## 1. 데이터 합치기
pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해서 합칠 수 있다.<br>
* pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사하다.
* pandas.concat은 하나의 축을 따라 객체를 이어 붙인다.
* combine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채운다.

### 1.1 데이터 베이스 스타일로 DataFrame 합치기
merge나 join 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합친다. pandas의 merge 함수를 이용해서 이런 알고리즘을 데이터에 적용할 수 있다.

**<표> merge 함수 인자 목록**

|     인자    |                                                                                           설명                                                                                           |
|:-----------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
|     left    |                                                                     merge하려는 DataFrame 중 왼쪽에 위치한 DataFrame                                                                     |
|    right    |                                                                    merge하려는 DataFrame 중 오른쪽에 위치한 DataFrame                                                                    |
|     how     |                                                             merge 방법, 'inner', 'outer', 'left', 'right', default는 'inner'                                                             |
|      on     |       merge하려는 칼럼, 반드시 두 DataFrame 객체 모두에 있는 칼럼이어야함. 만약 명시되지 않고 다른 join key도 주어지지 않으면 left와 right에서 공통되는 칼럼을 join key로 활용한다.      |
|   left_on   |                                                                          join key로 사용할 left DataFrame의 칼럼                                                                         |
|   right_on  |                                                                         join key로 사용할 right DataFrame의 칼럼                                                                         |
|  left_index |                                                           join key로 사용할 left DataFrame의 색인 로우 (다중 색인일 경우의 키)                                                           |
| right_index |                                                           join key로 사용할 right DataFrame의 색인 로우 (다중 색인일 경우의 키)                                                          |
|     sort    |                             join key에 따라 병합된 데이터를 사전 순으로 정렬, 기본 값은 True, 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수도 있다.                             |
|   suffixes  | 칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플, 기본 값은 ('_x', '_y'). 만약 'data'라는 칼럼 이름이 양쪽 DataFrame에 같이 존재하면 결과에서는 'data_x', 'data_y'로 나타난다. |
|     copy    |                                           False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본 값은 항상 복사가 이루어진다.                                           |

In [140]:
import pandas as pd
import numpy as np

In [141]:
df1 = pd.DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1' : range(7)})
df2 = pd.DataFrame({'key' : ['a', 'b', 'd'], 'data2' : range(3)})
print(df1)
print(df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


In [142]:
# 두 DataFrame에 공통된 칼럼이 있으므로 merge 함수를 쓸 때, 명시하지 않아도 되지만 명시하는 습관을 들이는 게 좋다.
# 결과는 같다.
print(pd.merge(df1, df2))
print(pd.merge(df1, df2, on = 'key'))

   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0
   data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0


In [143]:
# 두 객체에 공통되는 칼럼 이름이 하나도 없다면 아래와 같이 지정한다.
df3 = pd.DataFrame({'lkey' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1' : range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2' : range(3)})
print(df3)
print(df4)

   data1 lkey
0      0    b
1      1    b
2      2    a
3      3    c
4      4    a
5      5    a
6      6    b
   data2 rkey
0      0    a
1      1    b
2      2    d


In [144]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [145]:
# left join
print(df1)
print(df2)
pd.merge(df1, df2, on = 'key', how = 'left')

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b
   data2 key
0      0   a
1      1   b
2      2   d


,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [146]:
# right join
pd.merge(df1, df2, left_on = 'key', right_on = 'key', how = 'right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


In [147]:
# outer join
pd.merge(df1, df2, left_on = 'key', right_on = 'key', how = 'outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [148]:
# 여러개의 키로 merge하려면 칼럼 이름이 들어간 리스트를 넘긴다.
left = pd.DataFrame({'key1' : ['foo', 'foo', 'bar'],
                     'key2' : ['one', 'two', 'three'],
                     'lval' : [1, 2, 3]})
right = pd.DataFrame({'key1' : ['foo', 'foo', 'bar', 'bar'],
                      'key2' : ['one', 'one', 'one', 'two'],
                      'rval' : [4, 5, 6, 7]})
print(left)
print(right)

  key1   key2  lval
0  foo    one     1
1  foo    two     2
2  bar  three     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [149]:
pd.merge(left, right, on = ['key1', 'key2'], how = 'left')

,key1,key2,lval,rval
0,foo,one,1,4.0
1,foo,one,1,5.0
2,foo,two,2,NaN
3,bar,three,3,NaN


In [150]:
# 겹치는 칼럼이름을 처리하는 예제
pd.merge(left, right, left_on = 'key1', right_on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,three,3,one,6
5,bar,three,3,two,7


In [151]:
pd.merge(left, right, left_on = 'key1', right_on = 'key1', suffixes = ('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,three,3,one,6
5,bar,three,3,two,7


### 1.2 색인 머지하기
merge하려는 key가 DataFrame의 색인일 수 있다. 이 경우에는 left_index = True, right_index = True 옵션을 지정해 해당 색인을 머지키로 활용할 수 있다. DataFrame의 join 메서드를 활용하는 방법도 존재하며 join 메서드는 칼럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다.(left join 방법이 default)

In [152]:
left1 = pd.DataFrame({'key' : ['a', 'b', 'a', 'a', 'b','c'], 'value' : range(6)})
right1 = pd.DataFrame({'group_val' : [3.5, 7]}, index = ['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [153]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [154]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'left')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [155]:
# 계층 색인된 데이터의 경우
# 리스트로 여러개의 칼럼을 지정해서 merge해야하는 상황
lefth = pd.DataFrame({'key1' : ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2' : [2000, 2001, 2002, 2001, 2002],
                      'data' : np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape(6,2),
                      index = [['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                               [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns = ['event1', 'event2'])
print(lefth)
print(righth)

   data    key1  key2
0   0.0    Ohio  2000
1   1.0    Ohio  2001
2   2.0    Ohio  2002
3   3.0  Nevada  2001
4   4.0  Nevada  2002
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [156]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index = True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [157]:
# 양쪽에 공통으로 있는 여러 개의 색인을 머지하는 경우
left2 = pd.DataFrame([[1, 2], [3, 4], [5, 6]], index = ['a', 'c', 'e'], columns = ['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7, 8], [9, 10], [11, 12], [13, 14]], index = ['b', 'c', 'd', 'e'],
                      columns = ['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14


In [158]:
pd.merge(left2, right2, how = 'left', left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
c,3,4,9.0,10.0
e,5,6,13.0,14.0


In [159]:
# join 메서드 활용
left2.join(right2, how = 'left')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
c,3,4,9.0,10.0
e,5,6,13.0,14.0


In [160]:
# join 메서드를 호출한 DataFrame의 칼럼 중 하나에 대해서 조인을 수행하는 것도 가능하다.
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [161]:
left1.join(right1, on ='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [165]:
# 색인 대 색인으로 여러 DataFrame을 합치려면 그냥 간단히 머지하려는 DataFrame의 리스트를 join 메서드에 넘긴다. 하지만 
#일반적으로 이런 merge는 concat 메서드를 활용한다.
another = pd.DataFrame([[7, 8], [9, 10], [11, 12], [16, 17]], index = ['a', 'c', 'e', 'f'],
                       columns = ['New York', 'Oregon'])
print(left2)
print(right2)
print(another)

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14
   New York  Oregon
a         7       8
c         9      10
e        11      12
f        16      17


In [166]:
left2.join([right2, another], how = 'left')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9.0,10.0,9,10
e,5,6,13.0,14.0,11,12


### 1.3 축 따라 이어붙이기
데이터를 합치는 또 다른 방법으로넌 이어붙이기(concatenation), 연결(binding), 적층(stacking) 등이 있는 데, numpy는 ndarray를 연결하는 concatenate 함수를 제공한다.

#### 1.3.1 ndarray의 경우 

In [167]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [168]:
print(np.concatenate([arr, arr], axis = 0))
print(np.concatenate([arr, arr], axis = 1))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]


#### 1.3.2 Series 또는 DataFrame의 경우
Series나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게이어 붙일 수 있도록 되어있다.
pandas의 concat 함수를 이용하면 다음을 고려하여 데이터를 이어붙일 수 있다.<br>
* 만약에 연결하려는 두 객체의 색인이 서로 다르다면 결과는 그 색인의 교집합이어야하는가 아니면 합집합이어야 하는가?
* 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있는가?
* 어떤 축으로 연결할 것인지 고려해야 하는가?

**<표> concat 함수 인자**

| 인자             | 설명                                                                                                                                                                                                |
|------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| objs             | 이어붙일 pandas 객체의 사전이나 리스트, 필수 인자                                                                                                                                                   |
| axis             | 이어붙일 축 방향, 기본 값은 0                                                                                                                                                                       |
| join             | 조인 방식, 'inner'(내부 조인, 교집합)과 'outer'(외부조인, 합집합)가 있으며, 기본값은 'outer'                                                                                                        |
| join_axes        | 합집합/교집합을 수행하는 대신 다른 n-1 축으로 사용할 색인을 지정한다.                                                                                                                               |
| keys             | 이어붙일 객체나 이어붙인 축에 대한 계층 색인을 생성하는 데 연관된 값이다. 리스트나 임의의 값이 들어있는 배열, 튜플의 배열 또는 배열의 리스트(levels 옵션에 다차원 배열이 넘어온 경우)가 될 수 있다. |
| levels           | 계층 색인 레벨로 사용할 색인을 지정한다. keys가 넘어온 경우 여러 개의 색인을 지정한다.                                                                                                            |
| names            | keys나 levels 혹은 둘 다 있을 경우, 생성된 계층 레벨을 위한 이름                                                                                                                                    |
| verify_integrity | 이어붙인 객체에 중복되는 축이 있는 지 검사하고 있다면 예외를 발생시킨다. 기본 값은 False로, 중복을 허용한다.                                                                                        |
| ignore_index     | 이어붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인을 생성한다.                                                                                                                    |

In [169]:
s1 = pd.Series([0, 1], index = ['a', 'b'])
s2 = pd.Series([2, 3, 4], index = ['c', 'd', 'e'])
s3 = pd.Series([5, 6], index = ['f', 'g'])
print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [170]:
pd.concat([s1, s2 ,s3], axis = 0) # pandas의 Series 객체를 리스트형태로 나열해서 넘긴 경우

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [171]:
pd.concat([s1, s2 ,s3], axis = 1, join = 'outer') # default는 outer

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [172]:
pd.concat([s1, s2 ,s3], axis = 1, join = 'inner') # 로우 색인이 겹치는 부분이 없으므로

,0,1,2


In [173]:
s4 = pd.concat([s1 * 5, s3])
print(s1)
print(s4)

a    0
b    1
dtype: int64
a    0
b    5
f    5
g    6
dtype: int64


In [174]:
# 겹치는 로우 색인으로 inner(교집합)을 구하는 경우
pd.concat([s1, s4], join = 'inner', axis = 1)

,0,1
a,0,0
b,1,5


In [175]:
# join_axes 인자로 merge 하려는 색인 축을 직접 지정하는 경우
pd.concat([s1, s4], join_axes = [['a', 'c', 'b', 'e']], axis = 1)

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [176]:
# Series를 이어붙일 경우 개별 Series를 구분하기 위해 축에 대해 계층적 색인을 생성하는 경우 keys 인자를 활용한다.
print(s1)
print(s3)

a    0
b    1
dtype: int64
f    5
g    6
dtype: int64


In [177]:
result = pd.concat([s1, s1, s3], keys = ['one', 'two', 'three'], axis = 0)
result #long format

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [178]:
# 위에서 long format인 result를 wide format을 바꾼다.
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [179]:
# Series를 axis = 1로 병합하여 DataFrame으로 만들 경우 keys 인자의 값은 DataFrame 칼럼의 이름이 된다.
pd.concat([s1, s2, s3], axis = 1, keys = ['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [180]:
# DataFrame 객체에 대해서도 같은 방식으로 적용이 가능하다.
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ['a', 'b', 'c'], columns = ['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ['a' , 'c'], columns = ['three', 'four'])
print(df1)
print(df2)
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'])

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [181]:
# 리스트 대신 객체의 사전을 넘긴다면 사전의 키가 keys의 옵션으로 사용된다.
pd.concat({'level1' : df1, 'level2' : df2}, axis = 1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [182]:
res = pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'], names = ['upper', 'lower'])
res

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [183]:
# DataFrame의 로우 색인이 불필요한 경우에는 ignore_index = True 옵션을 활용한다.
df1 = pd.DataFrame(np.random.randn(3,4), columns = ['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2,3), columns = ['b', 'd', 'a'])
print(df1)
print(df2)

          a         b         c         d
0  0.348862  0.516009 -0.489821 -0.628746
1  0.266415  1.083685  0.303349 -1.107238
2 -0.976152 -2.132725  0.542522 -0.225276
          b         d         a
0  2.750476  1.703181 -0.491991
1 -1.021350 -0.274269 -0.237797


In [184]:
print(pd.concat([df1, df2], axis = 0)) # 기본값인 ignore_index = False 인경우
print(pd.concat([df1, df2], axis = 0, ignore_index = True))

          a         b         c         d
0  0.348862  0.516009 -0.489821 -0.628746
1  0.266415  1.083685  0.303349 -1.107238
2 -0.976152 -2.132725  0.542522 -0.225276
0 -0.491991  2.750476       NaN  1.703181
1 -0.237797 -1.021350       NaN -0.274269
          a         b         c         d
0  0.348862  0.516009 -0.489821 -0.628746
1  0.266415  1.083685  0.303349 -1.107238
2 -0.976152 -2.132725  0.542522 -0.225276
3 -0.491991  2.750476       NaN  1.703181
4 -0.237797 -1.021350       NaN -0.274269


### 1.4 겹치는 데이터 합치기
데이터를 합칠 때 merge나 이어붙이기로는 불가능한 상황이 있는데, 두 데이터셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그렇다. 벡터화된 if-else 구문을 표현하는 numpy의 where 함수를 통해서 자세히 알아보자.

In [185]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index = ['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype = np.float64), index = ['f', 'e', 'd', 'c', 'b', 'a'])
print(a)
print(b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64


In [186]:
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [187]:
pd.isnull(a)

f     True
e    False
d     True
c    False
b    False
a     True
dtype: bool

In [188]:
print(np.where(pd.isnull(a), b, a))

[ 0.   2.5  2.   3.5  4.5  nan]


In [189]:
# Series 객체의 combine_first라는 메서드는 위와 동일한 연산을 제공하며 데이터 정렬기능까지 제공한다.
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [190]:
# DataFrame에서 combine_first 메서드는 칼럼에 대해 같은 동작을 한다. 그래서 이를 통해 호출하는 객체에서 누락된 데이터를
# 인자로 넘긴 객체에 있는 값으로 채워 넣을 수 있다.
df1 = pd.DataFrame({'a' : [1, np.nan, 5, np.nan],
                    'b' : [np.nan, 2, np.nan, 6],
                    'c' : range(2, 18, 4)})
df2= pd.DataFrame({'a' : [5, 4, np.nan, 3, 7],
                   'b' : [np.nan, 3, 4, 6, 8]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [191]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [192]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 2. 재형성과 피벗
표 형식의 데이터를 재배치하는 다양한 기본 연산이 존재하는데. 이런 연산을 재형성 또는 피벗 연산이라고 한다. R에서 tidy data 개념을 차용한 reshape2 패키지나 tidyr 패키지와 같은 역할을 한다고 생각하면된다.(long format, wide format)

### 2.1 계층적 색인으로 재형성하기
계층적 색인은 다음과 같은 DataFrame의 데이터를 재배치하는한결같은 방식을 제공한다.
* stack : 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.(R에서의 melt, wide format -> long format으로 변환)
* unstack : 로우를 칼럼으로 피벗시킨다. (R에서의 dcast, long format -> wide format)

In [193]:
data = pd.DataFrame(np.arange(6).reshape(2,3),
                    index = pd.Index(['Ohio', 'Colorado'], name = 'state'),
                    columns = pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [194]:
result = data.stack()
print(result)
print(result.shape)

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32
(6,)


In [195]:
data = result.unstack()
print(data)
print(data.shape)

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5
(2, 3)


In [196]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [197]:
# 보통 MultiIndex 객체를 unstack 할 때, 가장 안쪽에 있는 Index의 값들을 칼럼으로 만드는 것이 default이지만 unstack 메서드의 
# level 옵션으로 지정할 수도 있다.
print(result.unstack(level = 'state'))
print(result.unstack(level = 0))

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5
state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5


In [198]:
print(result.unstack(level = 'number'))
print(result.unstack(level = 1))

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5
number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


In [199]:
s1 = pd.Series([0, 1, 2, 3], index = ['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index = ['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys = ['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [200]:
# 해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수 있다.
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [201]:
print(data2.unstack().stack())
print(data2.unstack().stack(dropna = False))

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64
one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64


In [202]:
df = pd.DataFrame({'left' : result, 'right' : result  + 5}, columns = pd.Index(['left', 'right'], name = 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [203]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [204]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [205]:
type(df.to_records())

numpy.recarray

### 2.2 pivoting으로 데이터 나열방식 바꾸기
* pivot : set_index를 사용해서 계층적 색인을 만들고 unstack 메서드를 이용해서 형태 변경하는 것과 동일한 연산

In [206]:
# pivot 메서드 활용
test = pd.DataFrame({'order' : [1,2,3,4], 'item' : ['a', 'b', 'b', 'c'], 'value' : [20, 10, 20, 10]})
test

,item,order,value
0,a,1,20
1,b,2,10
2,b,3,20
3,c,4,10


In [207]:
test.pivot('order', 'item', 'value')

item,a,b,c
order,,,
1,20.0,NaN,NaN
2,NaN,10.0,NaN
3,NaN,20.0,NaN
4,NaN,NaN,10.0


In [208]:
test.pivot('item', 'order', 'value')

order,1,2,3,4
item,,,,
a,20.0,NaN,NaN,NaN
b,NaN,10.0,20.0,NaN
c,NaN,NaN,NaN,10.0


## 3. 데이터 변형

### 3.1 중복 제거하기
기본적으로 아래의 두 메서드는 모든 칼럼에 적용되며 중복을 찾아내기 위한 부분집합을 따로 지정할 수 있다. keep 인자에 대한 값은 중복된 로우를 발견시 첫 번째 값을 유지할 것이냐 마지막 값을 유지할 것이냐에 대한 것이다.

| 메서드                                         | 설명                                                                  |
|------------------------------------------------|-----------------------------------------------------------------------|
| duplicated(..., keep = 'first' or 'last')      | 각 로우가 중복인 지 아닌 지를 알려주는 불리언 Series 객체를 반환한다. |
| drop_duplicates(..., keep = 'first' or 'last') | duplicated 배열이 False인 DataFrame을 반환한다.                       |

In [209]:
data = pd.DataFrame({'k1' : ['one'] * 3 + ['two'] * 4, 'k2' : [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [210]:
# DataFrame의 duplicated 메서드는 각 로우가 중복인지 아닌지를 알려주는 불리언 Series 객체를 반환한다.
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [211]:
# drop_duplicates() 메서드는 duplicated 배열이 False인 DataFrame을 반환한다.
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [212]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [213]:
data.drop_duplicates(['k1', 'k2'], keep = 'last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### 3.2 함수나 매핑 이용해 데이터 변형하기

In [214]:
data = pd.DataFrame({'food' : ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned veef', 'Bacon', 'pastrami',
                               'honey ham', 'nova lox'],
                     'ounces' : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned veef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [215]:
# 육류별 동물을 담고 있는 데이터 생성
meat_to_animal = {'bacon' : 'pig',
                  'pulled pork': 'pig',
                  'pastrami' : 'cow',
                  'corned beef' : 'cow',
                  'honey ham' : 'pig',
                  'nova lox' : 'salmon'}
meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

In [216]:
# Series의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있다.
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned veef,7.5,NaN
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 3.3 값 치환하기
fillna 메서드를 사용해서 누락된 값을 채우는 일은 일반적인 값 치환 작업이다. map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면, replace 메서드는 같은 작업에 대해서 좀 더 간단하고 유연한 방법을 제공한다.

In [217]:
data = pd.Series([1, -999, 2, -999, -1000, 3])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [218]:
# replace 메서드를 이용한 값의 치환
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [219]:
# 여러 개의 값을 한번에 치환할 경우
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [220]:
# 여러 개의 값을 값마다 다른 값을 치환하는 경우
print(data.replace([-999, -1000], [np.nan, 0]))
print(data.replace({-999 : np.nan, -1000 : 0})) # 사전이용

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64
0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64


###  3.4 축 색인 이름 바꾸기
Series의 값처럼 축 이름 역시 유사한 방식으로 함수나 새롭게 바꿀 값으로 이용해서 변형할 수 있다. 새로운 자료 구조를 만들지 않고 그 자리에서 바로 축 이름을 변경하는 것이 가능하다.

In [221]:
data = pd.DataFrame(np.arange(12).reshape(3,4), index = ['Ohio', 'Colorado', 'New York'],
                    columns = ['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [222]:
# 축 색인 객체에도 map 메서드가 존재한다.
data.index.map(str.upper)

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [223]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [224]:
# 원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메서드를 사용한다.
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [225]:
# rename 메서드의 경우 사전 형식의 객체를 이용하여 축 이름 중 일부만 변경하는 것도 가능하다.
data.rename(index = {'OHIO' : 'INDIANA'}, columns = {'three' : 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [226]:
# rename 메서드를 사용하면 DataFrame을 직접 복사해서 index와 columns의 속성을 갱신할 필요 없이 바로 변경할 수 있다.
# 원본데이터를 바로 변경하려면 inplace = True 옵션을 넘겨주면 된다.
data.rename(index = {'OHIO' : 'INDIANA'}, inplace = True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 3.5 개별화와 양자화
연속성 데이터는 종종 개별로 분할하거나 분석을 위해 그룹별로 나누기도 하는 데, 수업에 참여하는 학생 그룹 데이터가 있고, 나이대에 따라 분류한다고 가정하자.

In [227]:
# pandas의 cut 함수를 사용해서 그룹으로 나누는 예제
ages = [20, 22, 25, 27, 21 ,23 ,37, 31, 61, 45 ,41, 32]
bins = [18, 25 ,35, 60, 100]
cats = pd.cut(ages, bins) # pd.cut(ages bins, right = False)로 왼쪽이 닫힌 폐구간을 만들수도 있다.
cats # 해당 객체는 그룹의 이름이 담긴 배열이라고 생각하면된다.

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [228]:
type(cats) # Categorical 객체

pandas.core.categorical.Categorical

In [229]:
cats.codes # Categorical 객체는 codes 속성에 있는 ages 데이터에 대한 카테고리 이름을 categories라는 배열에 내부적으로 담고 있다.

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [230]:
cats.categories

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

In [231]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [232]:
# labales 옵션으로 그룹의 이름을 넘겨줄 수도 있다.
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [235]:
# cut 함수에 명시적으로 그룹의 경계값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터에서 최소값과 최대값을 기준으로
# 균등한 길이의 그룹을 자동으로 계산한다.
data = np.random.randn(20) # 표준정규분포에서 값 20개를 추출한다.
result = pd.cut(data, 4, precision = 2) # 데이터의 분산에 따라 각각의 그룹마다 개수가 달라지게 나눠진다.
#(일단 보면 그룹의 길이는 동등하다.)
pd.value_counts(result)

(-0.55, 0.47]     8
(-1.56, -0.55]    6
(0.47, 1.48]      4
(1.48, 2.49]      2
dtype: int64

In [236]:
# 분위수를 사용하는 qcut을 활용한다.(그룹간의 데이터의 개수를 동등하게 만들기 위해)
result1 = pd.qcut(data, 4)
pd.value_counts(result1)

(0.668, 2.487]        5
(-0.00347, 0.668]     5
(-0.711, -0.00347]    5
[-1.556, -0.711]      5
dtype: int64

In [237]:
# qcut을 사용하되 cut과 유사하게 분위수를 직접 지정하는 경우도 가능하다.
result2 = pd.qcut(data, [0, 0.25, 0.5, 0.75, 1]) # 1분위수, 2분위수, 3분위수로 끊은 경우
pd.value_counts(result2)

(0.668, 2.487]        5
(-0.00347, 0.668]     5
(-0.711, -0.00347]    5
[-1.556, -0.711]      5
dtype: int64

### 3.6 특이값 찾아내고 제외하기
배열연산을 수행할 때는 특이값(outlier)를 제외하거나 적당한 값으로 대체하는 것이 중요하다.

In [238]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.head()

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570


In [239]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [240]:
# DataFrame의 한 칼럼에서 절대값이 3을 초과하는 값을 찾아내는 코드
col = data.ix[:,3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [241]:
# DataFrame에서 어떤한 로우의 원소값이 하나라도 절대값이 3을 초과하는 값을 찾아낸는 코드
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [242]:
# DataFrame에서 절대값이 3을 넘어가는경우 절대값이 3이되도록 새로운 값을 대입하는 코드
data[np.abs(data) > 3] = np.sign(data) * 3
data.head()

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570


### 3.7 치환과 임의 샘플링
numpy.random.permutation 함수를 이용하면 Series나 DataFrame의 로우를 쉽게 임의의 순서대로 재배치할 수 있다. 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수에 넘기면 바뀐 순서가 담긴 정수 배열이 생성된다.

In [243]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [244]:
# ix 메서드 기반 색인에서 사용
df.ix[sampler,:]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [245]:
# DataFrame의 take 메서드에서 사용
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [246]:
# permutation을 활용한 sampling
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [247]:
# np.random.randint 사용
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size = 10)

In [248]:
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [249]:
draws = bag.take(sampler)
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4,  7])

### 3.8 표시자 / 더미 변수
pd.get_dummies : one of C coding을 해주는 함수

In [250]:
df = pd.DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1' : range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [251]:
pd.get_dummies(df['key'])

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [252]:
dummies = pd.get_dummies(df['key'], prefix = 'key')
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [253]:
pd.merge(df[['data1']], dummies, left_on = 'data1', right_index = True, how = 'left')

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [254]:
df[['data1']].join(dummies)

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [264]:
# get_dummies와 cut
values = np.random.rand(10)
values

array([ 0.65812709,  0.98538502,  0.84840457,  0.86590152,  0.17831618,
        0.502923  ,  0.91047191,  0.92555633,  0.57545221,  0.10905226])

In [265]:
bins = [0, 0.2, 0.6, 0.8, 1]

In [267]:
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,0.0


## 4. 문자열 다루기
### 4.1 문자열 객체 메서드
문자열을 다뤄야하는 대부분의 애플리케이션은 내장 문자열 메서드만으로도 충분하다.

**<표> 파이썬 내장 문자열 메서드**

| 인자                  | 설명                                                                                                                                                           |
|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| count                 | 문자열에서 겹치지 않는 부분 문자열의 개수를 반환한다.                                                                                                          |
| endswith, startswith  | 문자열이 주어진 접미사(접두사)로 끝날 경우 True를 반환한다.                                                                                                    |
| join                  | 문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.                                                                                                      |
| index                 | 부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 ValueError 예외가 발생한다.                                                                 |
| find                  | 첫 번째 부분 무자열의 첫 번째 글자의 위치를 반환한다. index와 유사하지만 부분 문자열이 없을 경우 -1을 반환한다.                                                |
| rfind                 | 마지막 부분 문자열의 첫 번째 글자의 위치를 반환한다. 부분 문자열이 없다면 -1을 반환한다.                                                                       |
| replace               | 문자열을 다른 문자열로 치환한다.                                                                                                                               |
| strip, rstrip, lstrip | 개행문자를 포함한 공백문자를 제거한다. lstrip은 문자열의 시작 부분에 있는 공백 문자만을 제거하고, rstrip은 문자열의 마지막 부분에 있는 공백 문자만을 제거한다. |
| split                 | 문자열을 구분자를 기준으로 부분 문자열의 리스트로 분리한다.                                                                                                    |
| lower, upper          | 각각 알파벳 문자를 소문자 혹은 대문자로 변환한다.                                                                                                              |
| ljust, rjust          | 문자열을 오른쪽 혹은 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 곳은 공백문자를 채워넣어 주어진 길이를 가지는 문자열을 반환한다.                 |

In [270]:
# 쉼표로 구분된 문자열은 split 메서드를 이용해서 분리할 수 있다.
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [271]:
# split 메서드는 종종 공백문자(줄바꿈문자 포함)를 제거하는 strip 메서드와 조합해서 사용하기도 한다.
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [274]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [277]:
'::'.join(pieces)

'a::b::guido'

In [278]:
# 부분 문자열 찾기
'guido' in val

True

In [279]:
val

'a,b, guido'

In [291]:
# index 메서드는 문자열을 찾지 못하면 error를 발생시킨다. (ValueError)
# 값이있으면 string 상에서 해당 문자열의 위치를 반환한다.
val.index(',')

1

In [292]:
# find 메서드는 문자열을 찾지 못하는 -1을 반환한다.
# 값이있으면 string 상에서 해당 문자열의 위치를 반환한다.
val.find(':')

-1

In [293]:
# count 메서드는 특정 부분 문자열이 몇 건 발견되었는지를 반환한다.
val.count(',')

2

In [295]:
# replace 메서드는 찾아낸 패턴을 다른 문자열로 치환한다.
val.replace(',', '::')

'a::b:: guido'

In [298]:
val.replace(',', '')

'ab guido'

### 4.2 정규표현식
파이썬에는 re 모듈이 내장되어 문자열에 대한 정규표현식을 처리한다. re 모듈 함수는 패턴 매칭, 치환, 분리의 세 가지로 나눌 수 있다. 물론 이 세 가지는 서로 연관되어 있으며, 정규표현식은 텍스트 안에 존재하는 패턴을 표현하고 이를 여러가지 다양한 목적으로 사용할 수 있도록 되어있다.

**<표> 정규표현식 메서드**

| 인자              | 설명                                                                                                                                                                     |
|-------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| findall, finditer | 문자열에서 겹치지 않는, 발견된 모든 패턴을 반환한다. findall 메서드는 모든 패턴의 리스트를 반환하고, finditer 메서드는 이터레이터를 통해 하나씩 반환한다.                |
| match             | 문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다. 일치하는 패턴이 있다면 match 객체를 반환하고 그렇지 않을 경우 None를 반환한다.               |
| search            | 문자열에서 패턴과 일치하는 내용을 검색하고 match 객체를 반환한다. match 메서드와는 달리 시작부터 일치하는 내용만 찾지않고 문자열 어디든 일치하는 내용이 있다면 반환한다. |
| sub, subn         | 문자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환한다. 대체 표현 문자열은 \1, \2,... 같은 기호를 사용해서 매치 그룹의 요소를 참조한다.  |

In [299]:
# example
import re
text = 'foo bar\t baz \tqux'

In [300]:
re.split('\s+', text) # 정규표현식이 먼저 컴파일되고 split 메서드가 실행된다.

['foo', 'bar', 'baz', 'qux']

In [306]:
# re.compile을 통해 정규표현식을 직접 컴파일하고 컴파일하여 얻은 정규표현식 객체를 재사용
# 같은 정규표현식을 다른 문자열에도 적용해야할 때, 이러한 방법으로 정규표현식 객체를 만들어 사용하면 cpu 사용량을 아낄 수 있다.
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [307]:
# findall 메서드는 일치하는 모든 부분의 문자열을 찾아준다.
re.findall('\s+', text)

[' ', '\t ', ' \t']

In [308]:
regex.findall(text)

[' ', '\t ', ' \t']

In [311]:
# search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다.
m = re.search('\s+', text)
text[m.start():m.end()]

' '

In [317]:
# match 메서드는 문자열의 시작 부분에서만 일치하는 것만 반환한다.
print(re.match('\s+', text)) # 고로 위 text는 없다.

None


In [320]:
# sub 메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.
regex.sub('-', text)

'foo-bar-baz-qux'

In [324]:
re.sub('\s+', '-', text)

'foo-bar-baz-qux'

In [326]:
# 복잡한 example
text = '''Dave dave@google.com Steve steve@gmail.com Rob rob@gmail.com Ryan ryan@yahoo.com'''

In [328]:
pattern = '[A-z]+@[A-z]+.[A-z]+'

In [329]:
re.findall(pattern, text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [333]:
# 각 패턴을 괄호로 묶어 쪼개서 찾는다.
# 패턴에 그룹이 있으므로 findall 메서드는 튜플의 목록을 반환한다.
pattern1 ='([A-z]+)@([A-z]+).([A-z]+)'
regex = re.compile(pattern1)
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [332]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [337]:
# \\1, \\2와 같은 기호를 사용하여 각 패턴의 그룹에 접근하는 예제
print(regex.sub('Username: \\1, Domain : \\2, Suffix : \\3', text))

Dave Username: dave, Domain : google, Suffix : com Steve Username: steve, Domain : gmail, Suffix : com Rob Username: rob, Domain : gmail, Suffix : com Ryan Username: ryan, Domain : yahoo, Suffix : com


### 4.3 pandas의 벡터화된 문자열 함수

**<표> 벡터화된 문자열 메서드**

| 메서드                | 설명                                                                                        |
|-----------------------|---------------------------------------------------------------------------------------------|
| cat                   | 선택적인 구분자와 함께 요소별로 문자열을 이어붙인다.                                        |
| contains              | 문자열이 패턴이나 정규표현식을 포함하는지를 나타내는 불리언 배열은 반환하다.                |
| count                 | 일치하는 패턴의 개수를 반환한다.                                                            |
| endswith, startswith  | 각 요소에 대해 x.endswith(pattern), x.startswith(pattern)와 동일한 동작을 한다.             |
| findall               | 각 문자열에 대해 일치하는 패턴/정규표현식의 전체 목록을 구한다.                             |
| get                   | i 번째 요소를 반환한다.                                                                     |
| join                  | Series의 각 요소를 주어진 구분자를 연결한다.                                                |
| len                   | 각 문자열의 길이를 구한다.                                                                  |
| lower, upper          | 대, 소문자로 변환한다. 각 요소에 대한 x.lower(), x.upper()와 같다.                          |
| match                 | 주어진 정규표현식으로 각 요소에 대한 re.match를 수행하여 일치하는 그룹을 리스트로 반환한다. |
| pad                   | 문자열의 좌,우 혹은 양쪽에 공백을 추가한다.                                                 |
| center                | pad(side = 'both')와 동일하다.                                                              |
| repeat                | 값을 복사한다. 예를 들어, s.str.repeat(3)는 각 문자열에 대한 x * 3과 동일하다.              |
| replace               | 패턴/정규표현식과 일치하는 내용을 다른 문자열로 치환한다.                                   |
| slice                 | Series 안에 있는 각 문자열을 자른다.                                                        |
| split                 | 정규표현식 혹은 구분자로 문자열을 나눈다.                                                   |
| strip, rstrip, lstrip | 각 요소의 개행문자를 포함한 공백 문자를 제거한다.                                           |

In [340]:
data = {'Dave' : 'dave@google.com', 'Steve' : 'steve@gmail.com', 'Rob' : 'rob@gmail.com', 'Wes' : np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [343]:
# 문자열과 정규표현식 메서드는 map 메서드를 사용해서 각 값에 적용할 수 있지만 NA 값을 만나면 오류가 발생한다. 이런 문제에 대처하기위해
# Series에는 NA 값을 건너뛰게하는 간결한 문자열 처리 메서드가 있다.
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [344]:
# IGNORECASE 같은 re 옵션과 함께 정규표현식도 사용할 수 있다.
data.str.findall('[A-z]+@[A-z]+.[A-z]+')

Dave     [dave@google.com]
Rob        [rob@gmail.com]
Steve    [steve@gmail.com]
Wes                    NaN
dtype: object

In [357]:
# 벡터화된 요소
matches = data.str.match('([A-z]+)@([A-z]+).([A-z]+)')
matches

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  from ipykernel import kernelapp as app


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [358]:
# 벡터화된 요소를 str.get을 이요하거나 str 속성의 색인을 이용하여 요소를 꺼낼 수 있다.
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [359]:
matches.str.get(0)

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

In [360]:
# 다음 문법으로 문자열을 잘라낼 수 있다.
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object